In [78]:
import pandas as pd
import numpy as np
import re

In [79]:
hyper_method_name = "Gpt-5.2_리샘플링"

gt = pd.read_csv("../output/csv/gpt5&gemini/251230_final_GT.csv", encoding="cp949")
answer = pd.read_excel(f"../output/csv/gpt5&gemini/{hyper_method_name}.xlsx") ## .drop(columns="Additional treatment")

In [80]:
gt = gt.sort_values(by=['Paper ID', 'Sample']).reset_index(drop=True)
answer = answer.sort_values(by=['Paper ID', 'Sample']).reset_index(drop=True)

In [81]:
gt[['Paper ID', 'Sample']] == answer[['Paper ID', 'Sample']][0:27]

,Paper ID,Sample
0,True,True
1,True,True
2,True,True
3,True,True
4,True,True
5,True,True
6,True,True
7,True,True
8,True,True
9,True,True


In [82]:
def text_processor(df): 
    # ## 'not found' 값 처리
    # df = df.replace("not found", np.nan)

    ## `Electrolyte concentration` 변수 처리
    df["Electrolyte concentration"] = df["Electrolyte concentration"].apply(
        lambda x: float(x[:-1]) if (type(x) == str) and ('M' in x or 'm' in x) else x
    )
        
    ## `Voltage range` 변수 처리
    df["Voltage range"] = df["Voltage range"].apply(
        lambda x: x.replace("V", "").replace("v", "").replace("?", "-").replace("--", "-") if x is not np.NaN else None
    )
    df["Voltage range min"] = df["Voltage range"].apply(
        lambda x: float(x.split('-')[0]) if isinstance(x, str) and '-' in x else None
    )
    df["Voltage range max"] = df["Voltage range"].apply(
        lambda x: float(x.split('-')[1]) if isinstance(x, str) and '-' in x else None
    )
    df = df.drop(columns=["Voltage range"])

    ## `Temperature` 변수 처리
    df["Temperature"] = df["Temperature"].apply(lambda x: 25 if x == "room temperature" else x)
        
    return df   

In [83]:
def combine_duplicated_cols(df):
    int_crate_cols = [
        col for col in df.columns
        if re.fullmatch(r"C-rate\s+[0-9]+", col)
    ]    
    
    int2float_map = {}
    for int_col in int_crate_cols:
        int2float_map[f"{int_col}"] = int_col[:7] + f"{float(int_col[7:])}"

    for int_col, float_col in int2float_map.items():
        if float_col in df.columns:
            df[int_col] = df[int_col].combine_first(df[float_col])
            df.drop(columns=float_col, inplace=True)
        df.rename(columns={int_col: float_col}, inplace=True)
    
    return df

In [84]:
def c_rate_func(x): 
    if "C-rate" in x: 
        return True 
    else: 
        return False 

In [85]:
gt = text_processor(gt)
answer = text_processor(answer)

In [86]:
gt = combine_duplicated_cols(gt)
answer = combine_duplicated_cols(answer)

In [87]:
Morphological_Properties_col = [
    'Particle size', 
    'Particle shape', 
    'Particle distribution', 
    'Coating layer characteristics', 
    'Crystal structure and lattice characteristics'
]

gt = gt.drop(columns=Morphological_Properties_col)
answer = answer.drop(columns=Morphological_Properties_col)

In [88]:
gt_c_rate = gt.columns[gt.columns.map(c_rate_func)].to_list()
answer_c_rate = answer.columns[answer.columns.map(c_rate_func)].to_list()

In [89]:
# other_c_rate_col = [
#     'C-rate 0.05',
#     'C-rate 5.0',
#     'C-rate 10.0',
#     'C-rate 20.0',
#     'C-rate 40.0',
#     'C-rate 0.33',
#     'C-rate None',
#     'C-rate 0.2',
#     'C-rate 0.3',
#     'C-rate 4.0',
#     'C-rate 6.0',
# ]

In [90]:
paper_id = gt["Paper ID"].unique()

In [91]:
sort_col = [
    'Paper ID',
    'Sample',
    'Li ratio',
    'Ni ratio',
    'Co ratio',
    'Mn ratio',
    'O ratio',
    'W ratio',
    'Commercial NCM used',
    'Lithium source',
    'Synthesis method',
    'Crystallization method',
    'Crystallization final temperature',
    'Crystallization final duration (hours)',
    'Doping',
    'Coating',
    'Active material to Conductive additive to Binder ratio',
    'Electrolyte salt',
    'Electrolyte concentration',
    'Electrolyte solvent',
    'Electrolyte solvent ratio',
    'Additive',
    'Loading density (mass loading of NCM)',
    'Voltage range min',
    'Voltage range max',
    'Temperature',
    'C-rate 0.05',
    'C-rate 0.1', 
    'C-rate 0.2', 
    'C-rate 0.5', 
    'C-rate 1.0', 
    'C-rate 2.0',
    'C-rate 5.0', 
    'C-rate 10.0', 
    'C-rate 20.0', 
    'C-rate 40.0',
    # 'C-rate None', 
    'C-rate 4.0', 
    'C-rate 6.0'
]

In [92]:
tf_col = [
    "Li ratio", 
    "Ni ratio", 
    "Co ratio", 
    "Mn ratio", 
    "O ratio", 
    "W ratio", 
    "Commercial NCM used",
    'Lithium source',
    "Crystallization final temperature", 
    "Crystallization final duration (hours)", 
    "Electrolyte salt", 
    "Electrolyte concentration",
    'Voltage range min',
    'Voltage range max',
    'Temperature',
    'C-rate 0.05',
    'C-rate 0.1', 
    'C-rate 0.2', 
    'C-rate 0.5', 
    'C-rate 1.0', 
    'C-rate 2.0',
    'C-rate 5.0', 
    'C-rate 10.0', 
    'C-rate 20.0', 
    'C-rate 40.0',
    # 'C-rate None', 
    'C-rate 4.0', 
    'C-rate 6.0'
]

In [93]:
def is_number(val):
    try:
        float(val)
        return True
    except:
        return False

# key column 정의
key_cols = ["Paper ID", "Sample"]

# 모든 열 모음 (비교 대상)
all_cols = set(gt.columns) | set(answer.columns)
compare_cols = [col for col in all_cols if col not in key_cols]

# 결과 저장
results = []

# 모든 논문 처리
all_papers = sorted(set(gt["Paper ID"]) | set(answer["Paper ID"]))

for pid in all_papers:
    gt_group = gt[gt["Paper ID"] == pid].reset_index(drop=True)
    ans_group = answer[answer["Paper ID"] == pid].reset_index(drop=True)

    max_len = max(len(gt_group), len(ans_group))

    def pad_df(df, target_len):
        pad_len = target_len - len(df)
        if pad_len <= 0:
            return df
        empty = pd.DataFrame({col: [None] * pad_len for col in df.columns})
        empty["Paper ID"] = pid
        return pd.concat([df, empty], ignore_index=True)

    gt_group = pad_df(gt_group, max_len)
    ans_group = pad_df(ans_group, max_len)

    for i in range(max_len):
        row_gt = gt_group.iloc[i]
        row_ans = ans_group.iloc[i]

        result_row = {
            "Paper ID": pid,
            "Sample": row_gt["Sample"] if pd.notna(row_gt["Sample"]) else row_ans["Sample"]
        }

        for col in compare_cols:
            val_gt = row_gt[col] if col in row_gt and col in gt.columns else None
            val_ans = row_ans[col] if col in row_ans and col in answer.columns else None

            try:
                # C-rate 열이고 둘 다 NaN이면, 결과도 NaN
                if "C-rate" in col and pd.isna(val_gt) and pd.isna(val_ans):
                    match = float('nan')
                elif pd.isna(val_gt) and pd.isna(val_ans):
                    match = True
                elif pd.notna(val_gt) and pd.notna(val_ans):
                    if is_number(val_gt) and is_number(val_ans):
                        match = abs(float(val_gt) - float(val_ans)) <= 0.001
                    else:
                        match = str(val_gt).strip() == str(val_ans).strip()

                else:
                    match = False

            except:
                match = False

            # 결과 저장 방식 변경
            if col in tf_col:
                result_row[col] = match
            else:
                result_row[col] = val_ans  # 숫자 컬럼이 아닌 경우는 answer 값 그대로 저장

        results.append(result_row)

# 결과 DataFrame
df_result = pd.DataFrame(results)

# 누락된 열 채우기 (혹시라도)
for col in tf_col:
    if col not in df_result.columns:
        df_result[col] = False

# 컬럼 순서 정렬
df_result = df_result[sort_col]

# 저장
df_result.to_csv(f"../output/csv/acc_{hyper_method_name}.csv", index=False)

In [94]:
gt.columns

Index(['Paper ID', 'Sample', 'Li ratio', 'Ni ratio', 'Co ratio', 'Mn ratio',
       'O ratio', 'W ratio', 'Commercial NCM used', 'Lithium source',
       'Synthesis method', 'Crystallization method',
       'Crystallization final temperature',
       'Crystallization final duration (hours)', 'Doping', 'Coating',
       'Active material to Conductive additive to Binder ratio',
       'Electrolyte salt', 'Electrolyte concentration', 'Electrolyte solvent',
       'Electrolyte solvent ratio', 'Additive',
       'Loading density (mass loading of NCM)', 'Temperature', 'C-rate 0.05',
       'C-rate 0.1', 'C-rate 0.2', 'C-rate 0.5', 'C-rate 1.0', 'C-rate 2.0',
       'C-rate 4.0', 'C-rate 5.0', 'C-rate 6.0', 'C-rate 10.0', 'C-rate 20.0',
       'C-rate 40.0', 'Voltage range min', 'Voltage range max'],
      dtype='object')

In [95]:
answer.columns

Index(['Paper ID', 'Sample', 'Li ratio', 'Ni ratio', 'Co ratio', 'Mn ratio',
       'O ratio', 'W ratio', 'Commercial NCM used', 'Lithium source',
       'Synthesis method', 'Crystallization method',
       'Crystallization final temperature',
       'Crystallization final duration (hours)', 'Doping', 'Coating',
       'Active material to Conductive additive to Binder ratio',
       'Electrolyte salt', 'Electrolyte concentration', 'Electrolyte solvent',
       'Electrolyte solvent ratio', 'Additive',
       'Loading density (mass loading of NCM)', 'Temperature', 'C-rate 0.05',
       'C-rate 0.1', 'C-rate 0.2', 'C-rate 0.5', 'C-rate 1.0', 'C-rate 2.0',
       'C-rate 4.0', 'C-rate 5.0', 'C-rate 6.0', 'C-rate 10.0', 'C-rate 20.0',
       'C-rate 40.0', 'Voltage range min', 'Voltage range max'],
      dtype='object')

In [96]:
df_result.columns

Index(['Paper ID', 'Sample', 'Li ratio', 'Ni ratio', 'Co ratio', 'Mn ratio',
       'O ratio', 'W ratio', 'Commercial NCM used', 'Lithium source',
       'Synthesis method', 'Crystallization method',
       'Crystallization final temperature',
       'Crystallization final duration (hours)', 'Doping', 'Coating',
       'Active material to Conductive additive to Binder ratio',
       'Electrolyte salt', 'Electrolyte concentration', 'Electrolyte solvent',
       'Electrolyte solvent ratio', 'Additive',
       'Loading density (mass loading of NCM)', 'Voltage range min',
       'Voltage range max', 'Temperature', 'C-rate 0.05', 'C-rate 0.1',
       'C-rate 0.2', 'C-rate 0.5', 'C-rate 1.0', 'C-rate 2.0', 'C-rate 5.0',
       'C-rate 10.0', 'C-rate 20.0', 'C-rate 40.0', 'C-rate 4.0',
       'C-rate 6.0'],
      dtype='object')